In this notebook the first treatments to the data of the datasets green trips and yellow trips are going to be carried out. The NTA pickup column will be added among others

In the end we will generate 2 new files with the treatment performed to share them and not have to run the entire treatment when we want to analyze the data later.

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [2]:
green = pd.read_csv("./data/green_trips_new_2.csv")
yellow = pd.read_csv("./data/yellow_trips_new.csv")
poligonos = gpd.read_file("./data/ny_nj_poligonos.geojson")
yellow2012 = pd.read_csv("data/yellow_tripdata_2012.csv")

In [3]:
print(green.shape)
green.head(2)

(3589048, 9)


,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount
0,2015-02-01 01:26:45,2015-02-01 01:49:58,-73.953545,40.811172,-73.984764,40.728386,1,8.11,27.8
1,2015-01-02 20:06:28,2015-01-02 20:14:04,-73.946709,40.714321,-73.961571,40.711475,1,1.29,9.8


In [4]:
print(yellow.shape)
yellow.head(2)

(7974059, 9)


,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount
0,2014-04-03 18:28:10,2014-04-03 18:54:32,-74.006033,40.706284,-73.918837,40.744946,1,8.70,34.8
1,2014-04-16 15:42:00,2014-04-16 16:07:00,-73.979558,40.749357,0.000000,0.000000,1,9.37,30.0


In [5]:
print(poligonos.shape)
poligonos.head(2)

(761, 8)


,boro_code,boro_name,county_fips,ntacode,ntaname,shape_area,shape_leng,geometry
0,3,Brooklyn,047,BK88,Borough Park,54005018.7472,39247.2280737,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ..."
1,4,Queens,081,QN51,Murray Hill,52488276.477,33266.904811,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ..."


In [6]:
# Geometry Green pickup coordenates
geometry_green_pick = [Point(xy) for xy in zip(green.pickup_longitude, green.pickup_latitude)]
crs = {'init': 'epsg:4326'}
geo_green = GeoDataFrame(green, crs=crs, geometry=geometry_green_pick)

In [7]:
green_nta = gpd.sjoin(geo_green, poligonos[["boro_name","ntacode","ntaname","geometry"]], how="left", op='intersects')
green_nta = green_nta.rename(columns={"boro_name":"pickup_boro","ntacode":"pickup_ntacode","ntaname":"pickup_ntaname","geometry":"p_geometry"})
green_nta = green_nta.loc[~green_nta.index.duplicated(keep='first')]
green_nta = green_nta.drop(["index_right"],axis=1)

In [8]:
green_nta.head()

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,p_geometry,pickup_boro,pickup_ntacode,pickup_ntaname
0,2015-02-01 01:26:45,2015-02-01 01:49:58,-73.953545,40.811172,-73.984764,40.728386,1,8.11,27.8,POINT (-73.95354 40.81117),Manhattan,MN09,Morningside Heights
1,2015-01-02 20:06:28,2015-01-02 20:14:04,-73.946709,40.714321,-73.961571,40.711475,1,1.29,9.8,POINT (-73.94671 40.71432),Brooklyn,BK90,East Williamsburg
2,2014-09-27 17:55:38,2014-09-27 18:19:56,-73.957626,40.718094,-73.947304,40.777813,5,6.12,26.3,POINT (-73.95763 40.71809),Brooklyn,BK73,North Side-South Side
3,2014-04-27 02:27:04,2014-04-27 02:39:02,-73.949501,40.713997,-73.987785,40.718582,2,3.68,17.3,POINT (-73.94950 40.71400),Brooklyn,BK73,North Side-South Side
4,2014-05-26 18:32:19,2014-05-26 18:44:13,-73.944092,40.672195,-73.977325,40.664013,1,2.40,11.5,POINT (-73.94409 40.67220),Brooklyn,BK61,Crown Heights North


In [9]:
# Geometry Green dropoff coordenates
geometry_green_drop = [Point(xy) for xy in zip(green.dropoff_longitude, green.dropoff_latitude)]
crs = {'init': 'epsg:4326'}
geo_green2 = GeoDataFrame(green_nta, crs=crs, geometry=geometry_green_drop)

In [19]:
green_nta2 = gpd.sjoin(geo_green2, poligonos[["boro_name","ntacode","ntaname","geometry"]], how="left", op='intersects')

In [20]:
green_nta2 = green_nta2.rename(columns={"boro_name":"dropoff_boro","ntacode":"dropoff_ntacode","ntaname":"dropoff_ntaname","geometry":"d_geometry"})
green_nta2 = green_nta2.loc[~green_nta2.index.duplicated(keep='first')]
green_nta2 = green_nta2.drop(["index_right"],axis=1)

In [21]:
print(green_nta2.shape)
green_nta2.head(3)

(3589048, 17)


,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,p_geometry,pickup_boro,pickup_ntacode,pickup_ntaname,d_geometry,dropoff_boro,dropoff_ntacode,dropoff_ntaname
0,2015-02-01 01:26:45,2015-02-01 01:49:58,-73.953545,40.811172,-73.984764,40.728386,1,8.11,27.8,POINT (-73.95354 40.81117),Manhattan,MN09,Morningside Heights,POINT (-73.98476 40.72839),Manhattan,MN22,East Village
1,2015-01-02 20:06:28,2015-01-02 20:14:04,-73.946709,40.714321,-73.961571,40.711475,1,1.29,9.8,POINT (-73.94671 40.71432),Brooklyn,BK90,East Williamsburg,POINT (-73.96157 40.71148),Brooklyn,BK73,North Side-South Side
2,2014-09-27 17:55:38,2014-09-27 18:19:56,-73.957626,40.718094,-73.947304,40.777813,5,6.12,26.3,POINT (-73.95763 40.71809),Brooklyn,BK73,North Side-South Side,POINT (-73.94730 40.77781),Manhattan,MN32,Yorkville


In [7]:
green_nta2 = pd.read_csv("data/green_nta.csv")

In [8]:
green_nta2 = green_nta2.rename(columns={"dropoff_boro":"dropoff_borough","pickup_boro":"pickup_borough"})
green_nta2 = green_nta2.dropna(how='any', subset=["pickup_borough"])

In [9]:
green_nta2.isna().sum()

Unnamed: 0               0
pickup_datetime          0
dropoff_datetime         0
pickup_longitude         0
pickup_latitude          0
dropoff_longitude        0
dropoff_latitude         0
passenger_count          0
trip_distance            0
total_amount             0
p_geometry               0
pickup_borough           0
pickup_ntacode           0
pickup_ntaname           0
d_geometry               0
dropoff_borough      11139
dropoff_ntacode      11139
dropoff_ntaname      11139
dtype: int64

In [10]:
green_nta2.to_csv("data/green_nta3.csv")

In [ ]:
#######################

In [25]:
# Geometry Yellow pickup coordenates
geometry_yellow_pick = [Point(xy) for xy in zip(yellow.pickup_longitude, yellow.pickup_latitude)]
crs = {'init': 'epsg:4326'}
geo_yellow = GeoDataFrame(yellow, crs=crs, geometry=geometry_yellow_pick)

In [26]:
yellow_nta = gpd.sjoin(geo_yellow, poligonos[["boro_name","ntacode","ntaname","geometry"]], how="left", op='intersects')
yellow_nta = yellow_nta.rename(columns={"boro_name":"pickup_boro","ntacode":"pickup_ntacode","ntaname":"pickup_ntaname","geometry":"p_geometry"})
yellow_nta = yellow_nta.loc[~yellow_nta.index.duplicated(keep='first')]
yellow_nta = yellow_nta.drop(["index_right"],axis=1)

In [27]:
yellow_nta.head()

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,p_geometry,pickup_boro,pickup_ntacode,pickup_ntaname
0,2014-04-03 18:28:10,2014-04-03 18:54:32,-74.006033,40.706284,-73.918837,40.744946,1,8.70,34.8,POINT (-74.00603 40.70628),Manhattan,MN25,Battery Park City-Lower Manhattan
1,2014-04-16 15:42:00,2014-04-16 16:07:00,-73.979558,40.749357,0.000000,0.000000,1,9.37,30.0,POINT (-73.97956 40.74936),Manhattan,MN20,Murray Hill-Kips Bay
2,2014-04-13 18:04:00,2014-04-13 18:07:00,-73.956453,40.775307,-73.954792,40.784992,1,0.89,6.5,POINT (-73.95645 40.77531),Manhattan,MN32,Yorkville
3,2014-05-21 19:33:00,2014-05-21 19:45:00,-73.987212,40.757850,-73.960198,40.775472,1,2.40,12.5,POINT (-73.98721 40.75785),Manhattan,MN17,Midtown-Midtown South
4,2014-05-30 16:28:00,2014-05-30 16:51:00,-73.974292,40.755397,-74.011867,40.704222,3,5.58,24.5,POINT (-73.97429 40.75540),Manhattan,MN19,Turtle Bay-East Midtown


In [28]:
# Geometry Yellow dropoff coordenates
geometry_yellow_drop = [Point(xy) for xy in zip(yellow.dropoff_longitude, yellow.dropoff_latitude)]
crs = {'init': 'epsg:4326'}
geo_yellow2 = GeoDataFrame(yellow_nta, crs=crs, geometry=geometry_yellow_drop)

In [29]:
yellow_nta2 = gpd.sjoin(geo_yellow2, poligonos[["boro_name","ntacode","ntaname","geometry"]], how="left", op='intersects')
yellow_nta2 = yellow_nta2.rename(columns={"boro_name":"dropoff_boro","ntacode":"dropoff_ntacode","ntaname":"dropoff_ntaname","geometry":"d_geometry"})
yellow_nta2 = yellow_nta2.loc[~yellow_nta2.index.duplicated(keep='first')]
yellow_nta2 = yellow_nta2.drop(["index_right"],axis=1)

In [36]:
print(yellow_nta2.shape)
yellow_nta2.head()

(7974059, 17)


,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,p_geometry,pickup_boro,pickup_ntacode,pickup_ntaname,d_geometry,dropoff_boro,dropoff_ntacode,dropoff_ntaname
0,2014-04-03 18:28:10,2014-04-03 18:54:32,-74.006033,40.706284,-73.918837,40.744946,1,8.70,34.8,POINT (-74.00603 40.70628),Manhattan,MN25,Battery Park City-Lower Manhattan,POINT (-73.91884 40.74495),Queens,QN31,Hunters Point-Sunnyside-West Maspeth
1,2014-04-16 15:42:00,2014-04-16 16:07:00,-73.979558,40.749357,0.000000,0.000000,1,9.37,30.0,POINT (-73.97956 40.74936),Manhattan,MN20,Murray Hill-Kips Bay,POINT (0.00000 0.00000),NaN,NaN,NaN
2,2014-04-13 18:04:00,2014-04-13 18:07:00,-73.956453,40.775307,-73.954792,40.784992,1,0.89,6.5,POINT (-73.95645 40.77531),Manhattan,MN32,Yorkville,POINT (-73.95479 40.78499),Manhattan,MN40,Upper East Side-Carnegie Hill
3,2014-05-21 19:33:00,2014-05-21 19:45:00,-73.987212,40.757850,-73.960198,40.775472,1,2.40,12.5,POINT (-73.98721 40.75785),Manhattan,MN17,Midtown-Midtown South,POINT (-73.96020 40.77547),Manhattan,MN40,Upper East Side-Carnegie Hill
4,2014-05-30 16:28:00,2014-05-30 16:51:00,-73.974292,40.755397,-74.011867,40.704222,3,5.58,24.5,POINT (-73.97429 40.75540),Manhattan,MN19,Turtle Bay-East Midtown,POINT (-74.01187 40.70422),Manhattan,MN25,Battery Park City-Lower Manhattan


In [37]:
yellow_nta2.isna().sum()

pickup_datetime           0
dropoff_datetime          0
pickup_longitude          0
pickup_latitude           0
dropoff_longitude         0
dropoff_latitude          0
passenger_count           0
trip_distance             0
total_amount              0
p_geometry                0
pickup_boro          168597
pickup_ntacode       168597
pickup_ntaname       168597
d_geometry                0
dropoff_boro         172401
dropoff_ntacode      172401
dropoff_ntaname      172401
dtype: int64

In [39]:
yellow_nta2.tail()

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,p_geometry,pickup_boro,pickup_ntacode,pickup_ntaname,d_geometry,dropoff_boro,dropoff_ntacode,dropoff_ntaname
7974054,2015-05-30 23:32:23,2015-05-30 23:55:59,0.000000,0.000000,0.000000,0.000000,2,4.40,22.80,POINT (0.00000 0.00000),NaN,NaN,NaN,POINT (0.00000 0.00000),NaN,NaN,NaN
7974055,2015-06-27 19:00:44,2015-06-27 19:03:34,-73.965103,40.806580,-73.970108,40.797161,1,0.64,5.30,POINT (-73.96510 40.80658),Manhattan,MN09,Morningside Heights,POINT (-73.97011 40.79716),Manhattan,MN12,Upper West Side
7974056,2015-06-21 17:59:31,2015-06-21 18:05:26,-73.996666,40.756790,-73.992065,40.747990,5,0.93,8.16,POINT (-73.99667 40.75679),Manhattan,MN13,Hudson Yards-Chelsea-Flatiron-Union Square,POINT (-73.99207 40.74799),Manhattan,MN17,Midtown-Midtown South
7974057,2015-04-10 19:28:54,2015-04-10 19:36:16,-73.982079,40.775158,-73.965836,40.800442,6,1.98,11.76,POINT (-73.98208 40.77516),Manhattan,MN14,Lincoln Square,POINT (-73.96584 40.80044),Manhattan,MN12,Upper West Side
7974058,2015-05-01 12:23:31,2015-05-01 12:30:06,-73.964752,40.759098,-73.974251,40.763218,1,0.60,8.16,POINT (-73.96475 40.75910),Manhattan,MN19,Turtle Bay-East Midtown,POINT (-73.97425 40.76322),Manhattan,MN17,Midtown-Midtown South


In [3]:
yellow_nta2 = pd.read_csv("data/yellow_nta.csv")

In [4]:
yellow_nta2 = yellow_nta2.rename(columns={"dropoff_boro":"dropoff_borough","pickup_boro":"pickup_borough"})
yellow_nta2 = yellow_nta2.dropna(how='any', subset=["pickup_borough"])

In [5]:
yellow_nta2.isna().sum()

Unnamed: 0               0
pickup_datetime          0
dropoff_datetime         0
pickup_longitude         0
pickup_latitude          0
dropoff_longitude        0
dropoff_latitude         0
passenger_count          0
trip_distance            0
total_amount             0
p_geometry               0
pickup_borough           0
pickup_ntacode           0
pickup_ntaname           0
d_geometry               0
dropoff_borough      23245
dropoff_ntacode      23245
dropoff_ntaname      23245
dtype: int64

In [6]:
yellow_nta2.to_csv("data/yellow_nta3.csv")